# Statistical Quality Control Analysis of Service Startup Times

This notebook presents a full simulation and control chart analysis for service startup durations (in milliseconds) using six different quality control techniques:
- x̄-R Chart
- Individual Chart
- Moving Average Chart
- Moving Range Chart
- EWMA Chart
- CUSUM Chart

Sample 10 includes an intentionally introduced assignable cause (outlier).


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


file_path = "SureVerisi.xlsx" 
df = pd.read_excel(file_path)


df.head()


,Obs_1,Obs_2,Obs_3,Obs_4,Obs_5,Sample_Mean,Sample_Range
0,3149,2959,3194,3457,2930,3137.8,527
1,2930,3474,3230,2859,3163,3131.2,615
2,2861,2860,3073,2426,2483,2740.6,647
3,2831,2696,3094,2728,2576,2785.0,518
4,3440,2932,3020,2573,2837,2960.4,867


In [ ]:
# 2. x̄-R Chart
x_bar_bar = means.mean()
r_bar = ranges.mean()
A2, D3, D4 = 0.577, 0.0, 2.114
ucl_x = x_bar_bar + A2 * r_bar
lcl_x = x_bar_bar - A2 * r_bar
ucl_r = D4 * r_bar
lcl_r = D3 * r_bar

plt.figure(figsize=(12, 6))
plt.plot(means, marker='o', label='Sample Means')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_x, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_x, color='red', linestyle='--', label='LCL')
plt.title('x̄ Chart')
plt.legend(); plt.grid(); plt.show()

plt.figure(figsize=(12, 6))
plt.plot(ranges, marker='s', label='Sample Ranges')
plt.axhline(r_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_r, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_r, color='red', linestyle='--', label='LCL')
plt.title('R Chart')
plt.legend(); plt.grid(); plt.show()


In [ ]:

# 3. Individual Chart
overall_std = means.std()
ucl_i = x_bar_bar + 3 * overall_std
lcl_i = x_bar_bar - 3 * overall_std

plt.figure(figsize=(12, 6))
plt.plot(means, marker='o', label='Individual Means')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_i, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_i, color='red', linestyle='--', label='LCL')
plt.title('Individual Chart')
plt.legend(); plt.grid(); plt.show()


In [ ]:

# 4. Moving Average Chart
window_size = 3
moving_avg = means.rolling(window=window_size).mean()

plt.figure(figsize=(12, 6))
plt.plot(moving_avg, marker='o', label='Moving Average')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_i, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_i, color='red', linestyle='--', label='LCL')
plt.title('Moving Average Chart')
plt.legend(); plt.grid(); plt.show()


In [ ]:

# 5. Moving Range Chart
moving_range = means.diff().abs()
mr_bar = moving_range[1:].mean()
d2 = 1.128
sigma_mr = mr_bar / d2
ucl_mr = 3 * sigma_mr

plt.figure(figsize=(12, 6))
plt.plot(moving_range, marker='o', label='Moving Range')
plt.axhline(mr_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_mr, color='red', linestyle='--', label='UCL')
plt.title('Moving Range Chart')
plt.legend(); plt.grid(); plt.show()


In [ ]:

# 6. EWMA Chart
alpha = 0.3
ewma = means.ewm(alpha=alpha, adjust=False).mean()
L = 3
ucl_ewma = x_bar_bar + L * overall_std * np.sqrt(alpha / (2 - alpha))
lcl_ewma = x_bar_bar - L * overall_std * np.sqrt(alpha / (2 - alpha))

plt.figure(figsize=(12, 6))
plt.plot(ewma, marker='o', label='EWMA')
plt.axhline(x_bar_bar, color='green', linestyle='--', label='CL')
plt.axhline(ucl_ewma, color='red', linestyle='--', label='UCL')
plt.axhline(lcl_ewma, color='red', linestyle='--', label='LCL')
plt.title('EWMA Chart')
plt.legend(); plt.grid(); plt.show()


In [ ]:

# 7. CUSUM Chart
k = 0.5 * overall_std
h = 5 * overall_std
cusum_pos, cusum_neg = [0], [0]

for i in range(1, len(means)):
    s_pos = max(0, cusum_pos[-1] + (means.iloc[i] - x_bar_bar - k))
    s_neg = min(0, cusum_neg[-1] + (means.iloc[i] - x_bar_bar + k))
    cusum_pos.append(s_pos)
    cusum_neg.append(s_neg)

plt.figure(figsize=(12, 6))
plt.plot(cusum_pos, label='CUSUM Positive', color='blue')
plt.plot(cusum_neg, label='CUSUM Negative', color='orange')
plt.axhline(h, color='red', linestyle='--', label='UCI')
plt.axhline(-h, color='red', linestyle='--', label='LCI')
plt.title('CUSUM Chart')
plt.legend(); plt.grid(); plt.show()
